## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips-1.csv"
file_type = "csv"

# CSV options
# infer_schema = "false"
# first_row_is_header = "false"
# delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header=True, inferSchema=True)
df.show()
# display(df)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[3]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
# Handling Categorical features
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
df_2 = indexer.fit(df).transform(df)
df_2.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [0]:
indexer = StringIndexer(inputCols=["sex", "smoker", "day", "time"], outputCols=["sex_indexed", "smoker_indexed", "day_indexed", "time_indexed"])
df_3 = indexer.fit(df).transform(df)
df_3.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
df_3.columns

Out[8]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
## Grouping independent features in a vector and output a single feature
from pyspark.ml.feature import VectorAssembler

feature_assembler = VectorAssembler(inputCols=['tip', 'sex_indexed', 'smoker_indexed', 'day_indexed', 'time_indexed', 'size'], outputCol='Independent Features')
output = feature_assembler.transform(df_3)

In [0]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,1.0,0.0,1.0...|
|[1.66,0.0,0.0,1.0...|
|[3.5,0.0,0.0,1.0,...|
|[3.31,0.0,0.0,1.0...|
|[3.61,1.0,0.0,1.0...|
|[4.71,0.0,0.0,1.0...|
|[2.0,0.0,0.0,1.0,...|
|[3.12,0.0,0.0,1.0...|
|[1.96,0.0,0.0,1.0...|
|[3.23,0.0,0.0,1.0...|
|[1.71,0.0,0.0,1.0...|
|[5.0,1.0,0.0,1.0,...|
|[1.57,0.0,0.0,1.0...|
|[3.0,0.0,0.0,1.0,...|
|[3.02,1.0,0.0,1.0...|
|[3.92,0.0,0.0,1.0...|
|[1.67,1.0,0.0,1.0...|
|[3.71,0.0,0.0,1.0...|
|[3.5,1.0,0.0,1.0,...|
|(6,[0,5],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
## Selecting the dependent and independent features 

final_df = output.select('Independent Features', 'total_bill')
final_df.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|[1.66,0.0,0.0,1.0...|     10.34|
|[3.5,0.0,0.0,1.0,...|     21.01|
|[3.31,0.0,0.0,1.0...|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|[4.71,0.0,0.0,1.0...|     25.29|
|[2.0,0.0,0.0,1.0,...|      8.77|
|[3.12,0.0,0.0,1.0...|     26.88|
|[1.96,0.0,0.0,1.0...|     15.04|
|[3.23,0.0,0.0,1.0...|     14.78|
|[1.71,0.0,0.0,1.0...|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|[1.57,0.0,0.0,1.0...|     15.42|
|[3.0,0.0,0.0,1.0,...|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|[3.92,0.0,0.0,1.0...|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|[3.71,0.0,0.0,1.0...|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|(6,[0,5],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression

train, test = final_df.randomSplit([0.75, 0.25])
reg = LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
reg = reg.fit(train)

In [0]:
reg.coefficients

Out[15]: DenseVector([3.2236, -0.7091, 3.0456, -0.334, -1.2196, 3.3671])

In [0]:
reg.intercept

Out[16]: 1.193901233636285

In [0]:
# predict on test data
preds = reg.evaluate(test)

In [0]:
preds.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,5],[1.25,2.0])|     10.07| 11.95764442557044|
|(6,[0,5],[1.25,2.0])|     10.51| 11.95764442557044|
| (6,[0,5],[2.0,3.0])|     16.31|17.742461044000933|
|(6,[0,5],[3.18,2.0])|     19.82|18.179115572405628|
|(6,[0,5],[3.27,2.0])|     17.78| 18.46923598857929|
|[1.01,1.0,0.0,1.0...|     16.99|10.140954670645877|
|[1.36,1.0,0.0,2.0...|     18.64|13.082747282702272|
|[1.5,0.0,0.0,3.0,...|     12.46|11.761595409096556|
|[1.57,0.0,0.0,1.0...|     15.42|12.655203995943218|
|[1.68,1.0,0.0,2.0...|     13.42|10.747140056558642|
|[1.71,0.0,0.0,1.0...|     10.27|13.106502421102247|
|[1.83,1.0,0.0,2.0...|     10.07|7.8635275998647645|
|[2.0,0.0,0.0,1.0,...|      8.77| 14.04133487321738|
|[2.0,0.0,0.0,1.0,...|     13.13| 14.04133487321738|
|[2.0,0.0,1.0,0.0,...|     13.81| 17.42093614951791|
|[2.0,0.0,1.0,1.0,...|     14.48|17.0869564623

In [0]:
preds.r2

Out[19]: 0.5008147276407152

In [0]:
preds.meanAbsoluteError, preds.meanSquaredError

Out[20]: (4.4273912855120425, 33.11781412860701)

In [0]:
# # Create a view or table

# temp_table_name = "tips-1_csv"

# df.createOrReplaceTempView(temp_table_name)

In [0]:
# %sql

# /* Query the created temp table in a SQL cell */

# select * from `tips-1_csv`

In [0]:
# # With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# # Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# # To do so, choose your table name and uncomment the bottom line.

# permanent_table_name = "tips-1_csv"

# # df.write.format("parquet").saveAsTable(permanent_table_name)